### 다음 웹툰정보 데이터 크롤링
* 평점순 웹툰 : http://m.webtoon.daum.net/data/mobile/webtoon?sort=score&page_size=9999&page_no=1
* 웹툰 이름, 장르, 작가, 게시일, url, 줄거리, 평점, 19금여부, 완결여부 크롤링

In [ ]:
from IPython.display import clear_output # clear_output() 으로 아웃풋 제거 가능
# 값 연속적으로 모두 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

import pandas as pd
import requests

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

In [ ]:
# 데이터를 가져와서 판다스의 데이터프레임으로 만드는 함수

def get_data(url):
    response = requests.get(url)
    json_info = response.json() # json -> dict 형식으로 알아서 바꿔줌
    
    return json_info

In [ ]:
# 19금 만화도 크롤링하려면 로그인을 해야한다

import getpass

myid = getpass.getpass('input your id : ')
mypassword = getpass.getpass('input your password : ')

LOGIN_URL = 'https://logins.daum.net/accounts/loginform.do'
LOGIN_DATA = {
    'id': myid,
    'inputPwd': mypassword
}
with requests.Session() as s:
    res = s.post(LOGIN_URL, data=LOGIN_DATA, verify=False, allow_redirects=False)

    print(res.status_code)

input your id : ··········
input your password : ··········
200


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
# 평점순 웹툰url
# url = 'http://m.webtoon.daum.net/data/mobile/webtoon?sort=score&page_size=50&page_no=1&1615108397290'

# -> page_size=9999(9999번까지) & page_no=1(1페이지로)
url1 = 'http://m.webtoon.daum.net/data/mobile/webtoon?sort=score&page_size=9999&page_no=1'
wbtn_raw1 = get_data(url1)

In [ ]:
wbt1_df = pd.DataFrame(columns = ['id', 'nickname', 'webtoonType', 'title', 'englishTitle', 'finishYn', 'titleImage2', 'thumbnailImage', 'thumbnailImage2', 'padThumbnailImage', 'artistCommentImage', 'homeThumbnailImage', 'appThumbnailImage', 'pcThumbnailImage', 'pcRecommendImage', 'pcHomeImage', 'pcHomeRightImage', 'pcHomeLeftColor', 'pcHomeRightColor', 'introduction', 'startDate', 'finishDate', 'restartDate', 'serviceStatus', 'viewerType', 'weekTerm', 'articleId', 'media', 'url', 'simpleUrl', 'webtoonGroupId', 'payYn', 'payType', 'price', 'ageGrade', 'restYn', 'monopolize', 'dateCreated', 'webtoonComment', 'cartoon', 'cp', 'webtoonWeeks', 'webtoonEpisodes', 'previewWebtoonEpisodes', 'latestWebtoonEpisode', 'webtoonServices', 'relatedProducts', 'promotionContents', 'score', 'tag', 'isNew', 'averageScore', 'seriesYn', 'ranking', 'diff', 'metricsScore', 'eventBanners', 'sort', 'sortWeight', 'gidamoo', 'webtoonNotice', 'couponAvailable'])

for data in wbtn_raw1['data']['webtoons']:
    wbt1_df.loc[len(wbt1_df)] = data

In [ ]:
wbt1_df2 = wbt1_df[['id', 'nickname', #웹툰 닉네임
                    'title', 'finishYn', 'thumbnailImage2', #'appThumbnailImage', 
                    #'viewerType', #(1206개중 1172개가 scroll) 'articleId', 'price' (1206개 다 0), 
                    'ageGrade', # (1206개중 1004개 0) #'restYn', # (1206개중 1153개가 'N') -> 휴재 여부인 듯
                    'dateCreated', 'cartoon', 'latestWebtoonEpisode', 'webtoonServices', 'averageScore'
                    # 'score',(1206개 다 0) #'isNew', # 신작 여부인 듯 'ranking', 'diff', 'metricsScore', 'sortWeight', 'couponAvailable'
        ]]

In [ ]:
# 썸네일 이미지 링크 정제

new_temp = pd.DataFrame(columns=['url'])
for data in wbt1_df2['thumbnailImage2']:
    new_temp.loc[len(new_temp)] = data['url']

In [ ]:
# 글/그림 작가 모으기

new_temp2 = pd.DataFrame(columns=['artist_name'])
for data in wbt1_df2['cartoon']:
    new_temp2.loc[len(new_temp2)] = data['artists'][0]['name']+'/'+data['artists'][1]['name']

In [ ]:
# 마지막 업로드 일자/ 현재(완료) 회차

new_temp3 = pd.DataFrame(columns=['count_chapter'])
new_temp4 = pd.DataFrame(columns=['latest_Created'])
for data in wbt1_df2['latestWebtoonEpisode']:
    try:
        new_temp3.loc[len(new_temp3)]= data['episode']
    except:
        new_temp3.loc[len(new_temp3)]= None
    try:
        new_temp4.loc[len(new_temp4)]= data['dateCreated']
    except:
        new_temp4.loc[len(new_temp4)]= None

In [ ]:
df = wbt1_df2[['id', 'nickname', 'title', 'finishYn', 'ageGrade', 'dateCreated', 'averageScore']]
df['artist']=new_temp2
df['latest_upload']=new_temp4
df['last_chapter']=new_temp3
df['img_url']=new_temp

In [ ]:
# 장르/카테고리, 소개글, 자동수집

intro = []
genres = []
categories = []
print('trying to getting datas from Daum Webtoon...', end='')
for i, data in df.iterrows():
    url2 = 'http://webtoon.daum.net/data/pc/webtoon/view/'+data['nickname']
    temp_df= get_data(url2)
    try:
        intro.append(temp_df['data']['webtoon']['introduction'])
    except:
        intro.append(None)
    gs=[]
    try:
        for data in temp_df['data']['webtoon']['cartoon']['genres']:
            gs.append(data['name'])
        gs_text = '/'.join(gs)
        genres.append(gs_text)
    except:
        genres.append(None)
    ls = []
    try:
        for data in temp_df['data']['webtoon']['cartoon']['categories']:
            ls.append(data['name'])
        tmp_text = '/'.join(ls)
        categories.append(tmp_text)
    except:
        categories.append(None)
    print(i,".", end='')
print("Crawling complete!")

df['intro'] = pd.Series(intro)
df['genres'] = pd.Series(genres)
df['categories'] = pd.Series(categories)

In [ ]:
# 웹툰 url

w_url = []
for i, data in df.iterrows():
    w_url.append('http://webtoon.daum.net/webtoon/view/'+data['nickname'])

df['page url'] = pd.Series(w_url)

In [ ]:
df.to_csv("D_Webtoon_1_toonsinfo.csv",index=False)